<a href="https://colab.research.google.com/github/Mohona77/Audio-Classification-SER-/blob/main/Imocap_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

# **Loading Base wav2vec2 model**

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
%%capture
!pip install gdown

In [5]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [6]:
%%capture
!pip install kaggle

In [7]:

%%capture
!pip install -U transformers[torch] datasets


%%capture
!pip install tensorboards

In [8]:
import os
import zipfile
from google.colab import files
import time
import json
import librosa
from datasets import Dataset, DatasetDict, Audio
import glob
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

print("Upload the kaggle.json file")
# Ensure files.upload() is called in the same cell as its import
files.upload() # Calling files.upload() immediately after import
time.sleep(5)

os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
time.sleep(5)

!kaggle datasets download -d dejolilandry/iemocapfullrelease
time.sleep(5)

with zipfile.ZipFile('iemocapfullrelease.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')
time.sleep(5)

!gdown --id 16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
time.sleep(5)

!unzip /content/IEMOCAP_splits.zip
time.sleep(5)

Upload the kaggle.json file


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/dejolilandry/iemocapfullrelease
License(s): unknown
100% 11.5G/11.5G [02:15<00:00, 44.2MB/s]
100% 11.5G/11.5G [02:15<00:00, 91.2MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16eCv6Sa4hklMoEk3ahp4lc1V561Elrml
To: /content/IEMOCAP_splits.zip
100% 817k/817k [00:00<00:00, 19.7MB/s]
Archive:  /content/IEMOCAP_splits.zip
   creating: iemocap/
   creating: iemocap/session5/
  inflating: iemocap/session5/test.json  
  inflating: iemocap/session5/train.json  
   creating: iemocap/session2/
  inflating: iemocap/session2/test.json  
  inflating: iemocap/session2/train.json  
   creating: iemoc

In [57]:
import json
import os

# Define the base path to the JSON files, assuming a structure like "/content/iemocap/sessionX/train.json"
json_base_path = "/content/IEMOCAP_full_release/iemocap"

# Function to update wav paths in a JSON file
def update_wav_paths(json_path, session_name):
    # Load JSON data
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Update each 'wav' path in the JSON data
    for entry in data:
        old_path = data[entry]["wav"]
        # Extract the filename from the old path
        file_name = old_path.split("/")[-1]

        # Create the new path starting from the session directory
        new_path = os.path.join(session_name, "dialog", "wav", file_name)
        data[entry]["wav"] = new_path  # Update the JSON entry with the new path

        # Optional: Print the updated path for verification
        print(f"Updated {entry}: {new_path}")

    # Save the updated JSON data back to the file
    with open(json_path, 'w') as f:
        json.dump(data, f, indent=4)

    print(f"File paths in {json_path} for {session_name} have been updated successfully!")

# Loop through each session directory and update both train and test JSON files
for session_num in range(1, 6):  # Assuming there are 5 sessions
    session_name = f"Session{session_num}"
    train_json_path = os.path.join(json_base_path, f"session{session_num}", "train.json")
    test_json_path = os.path.join(json_base_path, f"session{session_num}", "test.json")

    # Update paths in both train and test JSON files for the current session
    update_wav_paths(train_json_path, session_name)
    update_wav_paths(test_json_path, session_name)


Streaming output truncated to the last 5000 lines.
Updated Ses01F_script02_1_F015: Session5/dialog/wav/Ses01F_script02_1_F015.wav
Updated Ses01F_script02_1_F024: Session5/dialog/wav/Ses01F_script02_1_F024.wav
Updated Ses01F_script02_1_F025: Session5/dialog/wav/Ses01F_script02_1_F025.wav
Updated Ses01F_script02_1_M002: Session5/dialog/wav/Ses01F_script02_1_M002.wav
Updated Ses01F_script02_1_M007: Session5/dialog/wav/Ses01F_script02_1_M007.wav
Updated Ses01F_script02_1_M008: Session5/dialog/wav/Ses01F_script02_1_M008.wav
Updated Ses01F_script02_1_M009: Session5/dialog/wav/Ses01F_script02_1_M009.wav
Updated Ses01F_script02_1_M010: Session5/dialog/wav/Ses01F_script02_1_M010.wav
Updated Ses01F_script02_1_M011: Session5/dialog/wav/Ses01F_script02_1_M011.wav
Updated Ses01F_script02_1_M013: Session5/dialog/wav/Ses01F_script02_1_M013.wav
Updated Ses01F_script02_1_M014: Session5/dialog/wav/Ses01F_script02_1_M014.wav
Updated Ses01F_script02_1_M015: Session5/dialog/wav/Ses01F_script02_1_M015.wav
U

In [58]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/IEMOCAP_full_release/iemocap/session1/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/IEMOCAP_full_release/iemocap/session1/test.json",'r') as f:
    test_data = json.load(f)

In [59]:
   train_data

{'Ses02F_impro03_F000': {'wav': 'Session1/dialog/wav/Ses02F_impro03_F000.wav',
  'length': 4.0475736961451245,
  'emo': 'hap'},
 'Ses02F_impro03_F001': {'wav': 'Session1/dialog/wav/Ses02F_impro03_F001.wav',
  'length': 2.119954648526077,
  'emo': 'hap'},
 'Ses02F_impro03_F004': {'wav': 'Session1/dialog/wav/Ses02F_impro03_F004.wav',
  'length': 2.0,
  'emo': 'hap'},
 'Ses02F_impro03_F009': {'wav': 'Session1/dialog/wav/Ses02F_impro03_F009.wav',
  'length': 12.08702947845805,
  'emo': 'hap'},
 'Ses02F_impro03_F011': {'wav': 'Session1/dialog/wav/Ses02F_impro03_F011.wav',
  'length': 4.207210884353741,
  'emo': 'hap'},
 'Ses02F_impro03_F012': {'wav': 'Session1/dialog/wav/Ses02F_impro03_F012.wav',
  'length': 1.420907029478458,
  'emo': 'hap'},
 'Ses02F_impro03_F013': {'wav': 'Session1/dialog/wav/Ses02F_impro03_F013.wav',
  'length': 6.3349659863945575,
  'emo': 'hap'},
 'Ses02F_impro03_F014': {'wav': 'Session1/dialog/wav/Ses02F_impro03_F014.wav',
  'length': 4.569977324263038,
  'emo': 'hap

In [60]:
 test_data

{'Ses01F_script02_2_F000': {'wav': 'Session1/dialog/wav/Ses01F_script02_2_F000.wav',
  'length': 2.478140589569161,
  'emo': 'neu'},
 'Ses01F_script02_2_F001': {'wav': 'Session1/dialog/wav/Ses01F_script02_2_F001.wav',
  'length': 1.872517006802721,
  'emo': 'neu'},
 'Ses01F_script02_2_F006': {'wav': 'Session1/dialog/wav/Ses01F_script02_2_F006.wav',
  'length': 6.289977324263038,
  'emo': 'ang'},
 'Ses01F_script02_2_F008': {'wav': 'Session1/dialog/wav/Ses01F_script02_2_F008.wav',
  'length': 5.994512471655328,
  'emo': 'ang'},
 'Ses01F_script02_2_F009': {'wav': 'Session1/dialog/wav/Ses01F_script02_2_F009.wav',
  'length': 4.2,
  'emo': 'ang'},
 'Ses01F_script02_2_F010': {'wav': 'Session1/dialog/wav/Ses01F_script02_2_F010.wav',
  'length': 3.6105215419501135,
  'emo': 'ang'},
 'Ses01F_script02_2_F015': {'wav': 'Session1/dialog/wav/Ses01F_script02_2_F015.wav',
  'length': 10.49718820861678,
  'emo': 'sad'},
 'Ses01F_script02_2_F016': {'wav': 'Session1/dialog/wav/Ses01F_script02_2_F016.wav

In [61]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [62]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)

In [63]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 4446
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1085
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 1085
    })
})

In [64]:

%%capture
!git clone https://github.com/aalto-speech/mtkd4ser.git

In [74]:
import os

def add_base_path(example):
    # Assuming the original paths are relative to /content/iemocap/
    example['wav'] = os.path.join('/content/IEMOCAP_full_release/', example['wav'])
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

{'id': 'Ses02F_impro03_F000',
 'wav': '/content/IEMOCAP_full_release/Session1/dialog/wav/Ses02F_impro03_F000.wav',
 'length': 4.0475736961451245,
 'emo': 'hap'}

In [67]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [69]:
import os

file_path = '/content/IEMOCAP_full_release/Session1/dialog/wav/Ses02F_impro03_F000.wav'
print("File exists:", os.path.isfile(file_path))


File exists: False


In [70]:
print("Directory accessible:", os.access('/content/IEMOCAP_full_release/Session1/dialog/wav/', os.R_OK))


Directory accessible: True


In [71]:
import soundfile as sf

try:
    data, sample_rate = sf.read(file_path)
    print("Audio loaded successfully!")
except Exception as e:
    print("Error loading audio:", e)


Error loading audio: Error opening '/content/IEMOCAP_full_release/Session1/dialog/wav/Ses02F_impro03_F000.wav': System error.


In [73]:
!pip install pydub
from pydub import AudioSegment
try:
    audio = AudioSegment.from_wav(file_path)
    print("Audio loaded successfully!")
except Exception as e:
    print("Error loading audio:", e)


Error loading audio: [Errno 2] No such file or directory: '/content/IEMOCAP_full_release/Session1/dialog/wav/Ses02F_impro03_F000.wav'


In [68]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/4446 [00:00<?, ? examples/s]

FileNotFoundError: [Errno 2] No such file or directory: '/content/IEMOCAP_full_release/Session1/dialog/wav/Ses02F_impro03_F000.wav'